In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-40715?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Apply for Software Engineer II, ITC

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware Engineer II, ITCKarnataka, IndiaBecome a Part of the NIKE, Inc. TeamNIKE, Inc. doe

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [5]:
from langchain_core.output_parsers import JsonOutputParser

json_parser=JsonOutputParser()
json_res=json_parser.parse(res.content)
json_res

{'role': 'Software Engineer II, ITC',
 'experience': '2-5 years',
 'skills': ['Data engineering',
  'Python',
  'Java',
  'SQL',
  'Data Bricks',
  'Airflow',
  'EMR',
  'Hive',
  'Spark',
  'Microservices architecture',
  'Domain driven design',
  "RESTful API's",
  'Data management fundamentals',
  'Data storage principles',
  'CI/CD',
  'DevOps',
  'Cloud services',
  'AWS',
  'Marketing technology'],
 'description': 'We are looking for a Software engineer, to be a part of Nike’s Marketing Technology organization. This position will play a critical role, in developing Nike’s Customer Data platform, which supports and promotes our products and provides a unique user experience to all our customers in 166 countries globally.'}

In [6]:
type(json_res)

dict

In [7]:
import pandas as pd

df=pd.read_csv("My_Certificates_Links.csv")
df

,Techstack,Links
0,"Agile, Jenkins, Continuous Integration (CI), J...",Accenture Coding: Development & Advanced Engin...
1,"Machine Learning, ML, AI, Data Cleaning, Data ...",Coursera IBM Exploratory Data Analysis for Mac...
2,"Machine Learning, Supervised Learning, Linear ...",Coursera IBM Supervised Machine Learning: Regr...
3,"Data Structures, Java, Algorithms, Search Algo...",Udemy Java Data Structures & Algorithms + LEET...
4,"Chatbot, Gemini, Gen AI",Udemy Master Course in Chatbot Technology and...
5,Java,Java (Basic) https://www.hackerrank.com/certif...
6,"C++, Object-Oriented Programming (OOP)",OOPs Concepts in C++ https://verify.mygreatlea...
7,AWS,AWS S3 Basics https://www.coursera.org/account...
8,SQL Intermediate,SQL (Intermediate) https://www.hackerrank.com/...
9,SQL Basic,SQL Basic https://www.hackerrank.com/certific...


In [8]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [17]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'Coursera IBM Exploratory Data Analysis for Machine Learning https://www.coursera.org/account/accomplishments/records/627LU1OXN31D '},
  {'links': 'Udemy Java Data Structures & Algorithms + LEETCODE Exercises https://www.udemy.com/certificate/UC-92e02df2-e456-46ca-8146-c957b9137155/ '}],
 [{'links': 'Python for Machine Learning https://www.mygreatlearning.com/certificate/RRCZUIVA '},
  {'links': 'Introduction to Artificial Intelligence (AI) https://www.coursera.org/account/accomplishments/certificate/SAAZAE232E2P '}],
 [{'links': 'Java (Basic) https://www.hackerrank.com/certificates/1f9b59a2195e '},
  {'links': 'Udemy Java Data Structures & Algorithms + LEETCODE Exercises https://www.udemy.com/certificate/UC-92e02df2-e456-46ca-8146-c957b9137155/ '}],
 [{'links': 'SQL Basic  https://www.hackerrank.com/certificates/569eac076500 '},
  {'links': 'SQL (Intermediate) https://www.hackerrank.com/certificates/e8a8d9520f8c'}],
 [{'links': 'Udemy Java Data Structures & Algorithms + LE

In [15]:
job=json_res
job['skills']

['Data engineering',
 'Python',
 'Java',
 'SQL',
 'Data Bricks',
 'Airflow',
 'EMR',
 'Hive',
 'Spark',
 'Microservices architecture',
 'Domain driven design',
 "RESTful API's",
 'Data management fundamentals',
 'Data storage principles',
 'CI/CD',
 'DevOps',
 'Cloud services',
 'AWS',
 'Marketing technology']

In [19]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Samarth Narendra Bedare, a job applicant interested in the role mentioned above. You are applying to a company for a position 
        that aligns with your skills and experiences. Write an email expressing your interest in the job, showcasing your background 
        and how your skills can contribute to the company's goals. 
        Mention relevant experiences and achievements that make you a strong candidate for this position. 
        Additionally, include the following links to certifications that demonstrate your proficiency in the skills required for the role:
        {link_list}

        ### CANDIDATE DETAILS:
        - **Name**: Samarth Narendra Bedare
        - **Email**: bedaresamarth@gmail.com
        - **Phone**: (+91) 8793634732
        - **Skills**: Java, Python, C#, C++, MySQL, MS SQL Server, MS Visual Studio, Next.js, HTML, CSS, Javascript, MERN, Flask, Django, Web Development, Android, Desktop App Development, AI, ML, Deep Learning, Teamwork and Collaboration, Leadership and Project Management, Time Management, Good Communication.
        - **Education**:
            - **B.E in Computer Engineering** | Dr. D. Y. Patil Institute of Technology, Pimpri, Pune (Pursuing, Third Year, CGPA: 9.52, 2023-2026).
            - **Diploma in Computer Engineering** | Marathwada Mitra Mandal’s Polytechnic, Pune (Final Year Percentage: 91.31%, 2020-2023).
        - **Academic Projects**:
            - **Automated Fracture Detection System**: Developed a machine learning model with 90%+ accuracy for automated fracture detection in X-rays, reducing manual work by 80% and improving diagnostic efficiency by 40%. Role: Developer.
            - **Blog Full Stack Website**: Developed a blog website using the Next.js-enhanced MERN stack, enabling 100+ users for seamless content creation. Role: Developer. Link: [newspiral.vercel.app](http://newspiral.vercel.app).
            - **College Voting Portal**: Developed a secure desktop app for online college voting. Role: Team Lead & Developer. Technologies: MS Visual Studio, MS SQL Server.
        - **Position of Responsibility**:
            - **Technical Head** | Dr. D. Y. Patil Institute of Technology (Aug 2024).
            - **R&D Club Member** | Dr. D. Y. Patil Institute of Technology (Aug 2024 - Present).
        - **Extracurricular / Achievements**:
            - 1st rank in Diploma with 91.31%.
            - IBM Machine Learning Professional Certificate on Coursera (In Progress).
            - Java Data Structures & Algorithms Certificate on Udemy (Completion Date: June 2024).
            - Accenture NA Coding: Development & Advanced Engineering Job Simulation – (Nov 2024).
            - Published Research Paper on “College Voting Portal” (IRJMETS).
            - 3 ongoing Copyrights and Patents.
        - **Social Links**:
            - **GitHub**: [github.com/samarthbedare](https://github.com/samarthbedare)
            - **LinkedIn**: [linkedin.com/in/samarthbedare](https://linkedin.com/in/samarthbedare)

        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)


Subject: Application for Software Engineer II, ITC at Nike

Dear Hiring Manager,

I am excited to apply for the Software Engineer II, ITC position at Nike, as advertised. With a strong foundation in computer engineering and a passion for developing innovative solutions, I am confident that my skills and experience make me an ideal candidate for this role.

As a third-year student pursuing a B.E in Computer Engineering at Dr. D. Y. Patil Institute of Technology, I have developed a solid understanding of programming languages, including Java, Python, C#, and C++. My experience with MySQL, MS SQL Server, and MS Visual Studio has equipped me with the skills to design and develop efficient databases and software applications. Additionally, my proficiency in Next.js, HTML, CSS, and Javascript has enabled me to create dynamic web applications.

My academic projects demonstrate my ability to work on complex problems and develop innovative solutions. For instance, I developed an Automated Fract